In [1]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

In [11]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message.content)

Functions calling back,  
Layers deep in logic's dance,  
Endless loops of thought.


# Create question-answer pairs

From news scraped from Yahoo Finance and stored in json format, use gpt-4o-mini to create question-answer pairs.

For each piece of news, create 3 pairs.

In [2]:
import json

news_json = json.loads(open("news.json").read())
news_json

[{'title': 'Northvolt Founder’s Next Big Green Bet Has a Power Problem',
  'content': '(Bloomberg) -- After a four-decade career teaching soldiers to fire grenades, machine guns and howitzers, Lars Hjerpe was looking forward to a quiet retirement in the untamed woods of Sweden’s far north. Instead, he got a front-row seat to one of the world’s biggest experiments in green industrialization. Most Read from Bloomberg It’s been loud. Trucks regularly swirl up large dust clouds in front of Hjerpe’s idyllic red cottage, where he’d dreamed of peacefully tending to the land and feeding his hens, as workers erect the world’s first large-scale green steel plant across the street. The megafactory is part of a $100 billion effort to clean up some of the world’s most polluting sectors. If it works, an area bigger than England near the Arctic Circle could become a key European supplier of everything from green steel to climate-friendly batteries, fuels and fertilizers. The disruption has caused ang

In [9]:
system_msg = f"""
You are a helpful assistant. 
"""

user_msg = f"""
Here is the format of the json array of news you will be working with:
[
	{{
		"title": "The title of the news",
		"content: ["The content of the page"],
		"date": "Date of upload in UTC+0",
		"time": "Time of upload in UTC+0",
		"link": "Link to the page"
	}},
	{{
		"title": "The title of the news",
		"content: ["The content of the page"],
		"date": "Date of upload in UTC+0",
		"time": "Time of upload in UTC+0",
		"link": "Link to the page"
	}},...
]

You are given an array of news in json format. Here is the array in json:
```
{news_json}
```

For each news in the json array, I want you to create 3 question-answer pairs based on the title and content of the news. To create such pairs, you can follow these guidelines:
===
1. Preprocess the Article: First, you need to preprocess the article to extract relevant information. This typically includes:
- Splitting the article into sentences or paragraphs.
- Identifying key entities and facts, such as people, places, dates, events, and important concepts.
- Optionally, perform part-of-speech tagging or named entity recognition (NER) to assist in generating meaningful questions.

2. Identify Important Information: 
Determine the key pieces of information from the article that can be turned into questions. Some examples of important > information include:
- Who did something? (Person-related facts)
- What happened? (Events)
- When did something occur? (Dates and time)
- Where did something happen? (Places)
- Why did something happen? (Explanations or reasons)
- How did something happen? (Processes or sequences)
By identifying these details, you can focus on producing questions that revolve around these key facts.

3. Generate Questions
Aim to generate a variety of question types:
- Factual: "When did X happen?"
- Definition: "What is X?"
- Causal: "Why did X happen?"
- Procedural: "How does X work?"
- Inference: "What can be inferred about X?"

4. Generate answer for each question.
5. Structure the output in json format like this:
[
	{{
		"link": "Link to the news that is used to create question-answer pair",
		"date": "Date of upload of the news used to created question-answer pair"
		"context": "The context used to generate question-answer pair"
		"question": "The question created"
		"answer": "The answer for the questions created".
	}},
	...
]
===

For each piece of news, I want you to create 3 question-answer pairs based on the title and content of the news. Here is an example output:
[
    {{
		"link": "https://finance.yahoo.com/news/market-apos-too-optimistic-apos-175752296.html?",
		"date": "2024-10-08",
        "context": "Marie Curie won the Nobel Prize in Physics in 1903.",
        "question": "When did Marie Curie win the Nobel Prize in Physics?",
        "answer": "1903"
    }},
    {{
		"link": "https://finance.yahoo.com/news/mfadsfqwefsdfsa-175752296.html?",
		"date": "2024-10-08",
        "context": "The capital of France is Paris.",
        "question": "What is the capital of France?",
        "answer": "Paris"
    }}
]

Only output in the format specified.
"""

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg}
]

In [10]:
completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages)

from datetime import datetime

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
questions_file_name = f"questions_{current_time}.json"
with open(questions_file_name, "w") as f:
    f.write(completion.choices[0].message.content)

# Checking created question-answer pairs

In [16]:
question_answer_pairs = json.loads(open(questions_file_name).read())

check_system_msg = f"""
You are a helpful assistant. 
"""

check_user_msg = f"""
You are given a json array of question-answer pairs. Here is the format of the json array of news you will be working with:
[
	{{
		"link": "Link to the news that is used to create question-answer pair",
		"date": "Date of upload of the news used to created question-answer pair"
		"context": "The context used to generate question-answer pair"
		"question": "The question created"
		"answer": "The answer for the questions created".
	}},
	...
]

Here is the array in json:
{question_answer_pairs}

For each question-answer pair, you should check if the answer is correct. If the answer is correct, you should respond with "Correct". If the answer is incorrect, you should respond with "Incorrect" and add the correct asnwer.
To check the answer, you can follow these guidelines:
===
1. Read the context and question. You can also refer to the link provided to get more information.
2. Determine if the answer provided is correct based on the context.
3. If the answer is correct, respond with "Correct". If the answer is incorrect, respond with "Incorrect" and provide the correct answer.
4. Repeat the process for each question-answer pair.
5. Structure the output in json format like this:
[
    {{
        "question": "The question created",
        "answer": "The answer for the questions created",
        "response": "Correct" or "Incorrect",
        "correct_answer": "Correct answer if the response is Incorrect"
    }},
    ...
]
===

Here is the example output:
[
    {{
        "question": "When did Marie Curie win the Nobel Prize in Physics?",
        "answer": "1903",
        "response": "Correct"
    }},
    {{
        "question": "What is the capital of France?",
        "answer": "Washington",
        "response": "Incorrect",
        "correct_answer": "Paris"
    }},
]


Only output in the format specified.
"""

check_messages = [
    {"role": "system", "content": check_system_msg},
    {"role": "user", "content": check_user_msg}
]

In [17]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=check_messages
)

from datetime import datetime

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
check_questions_file_name = f"check_questions_{current_time}.json"
with open(check_questions_file_name, "w") as f:
    f.write(completion.choices[0].message.content)

# Misc

In [6]:
question_json = json.loads(open(questions_file_name).read())
print(len(question_json))

# check_json = json.loads(open(check_questions_file_name).read())
# print(len(check_json))

30


In [2]:
import json

final_quest = json.loads(open("./questions/final_questions.json").read())
print(len(final_quest))

links = list(set([qa["link"] for qa in final_quest]))
with open("links.txt", "w") as f:
    f.write("\n".join(links))

200


# Generate answers

In [22]:
questions_file_name = "questions_20241025_183804.json"
question_answer_pairs = json.loads(open(questions_file_name).read())

questions = [ qa["question"] for qa in question_answer_pairs]
questions

['When did the Federal Reserve cut interest rates for the first time since the COVID-19 pandemic?',
 'What was the average rate of the standard 30-year fixed-rate mortgage as of September 19, 2024?',
 'What do some experts believe falling interest rates could entice homeowners to do?',
 "What is the new total amount of Brazil's spending freeze for this year?",
 "What was President Lula's initial goal regarding the budget for 2024?",
 "What is the sentiment of investors towards Finance Minister Fernando Haddad's efforts?",
 'How has Warren Buffett performed compared to the S&P 500?',
 "What is the aggregate return of Warren Buffett's company's Class A shares?",
 "What has been Buffett's investment activity in the last seven quarters according to his 13Fs?",
 "What did Elon Musk's social media platform X do in Brazil?",
 "What did Brazil's top court order in late August regarding the platform X?",
 'What caused the shutdown of platform X in Brazil?',
 "Who was hired as CEO of Sumitomo's 

In [ ]:
import requests

def get_info_from_rag(question: str):
    print(question)
    prompt_template = f"{question} (And put the urls in the result. I want the result always has all urls you used for query)"
    response = requests.post(
        f"http://127.0.0.1:5555/rags",
        json={"text": f"{prompt_template}"},
    )
    
    return response.json()


current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
bot_responses_file_name = f"bot_responses_{current_time}.json"
with open(bot_responses_file_name, "w") as f:
    for question in questions:
        rag_info = get_info_from_rag(question)
        rag_response = rag_info["data"]
        f.write(json.dumps(rag_response))